In [1]:
import requests
import json
from PIL import Image
import base64
import os
http_url = 'https://api-us.faceplusplus.com/facepp/v3/detect'
key = "7AUVsIF0R8T2YYZmrfe5-4r9FdumOFYc"
secret = "0TF9Xcz_9RVi10q_2Y6SqKdkiM8Fmlze"
image_path = "pic5.jpg"
image_name= os.path.splitext(image_path)[0]
# im = Image.open(filepath)
with open(image_path, "rb") as img_file:
    img_string = base64.b64encode(img_file.read())
# print(img_string)
response = requests.post(
    'https://api-us.faceplusplus.com/facepp/v3/detect',
    {
        'api_key': key,
        'api_secret': secret,
        # 'image_url': img_url,
#         'image_base64': 'iop.jpg',
        'image_base64': img_string,
        'return_landmark': 1,
        'return_attributes': 
'headpose,eyestatus,facequality,mouthstatus,eyegaze,blur,emotion'
    }
)
data = response.json()
print(data)
json_file=image_name + ".json"
with open(image_name + ".json", 'w') as outfile:
    json.dump(data, outfile)


{'request_id': '1606638989,2dd733d1-acb9-4e10-8932-331f56d3f277', 'time_used': 93, 'faces': [{'face_token': 'dc76fe01647f856938ef3899da04be08', 'face_rectangle': {'top': 74, 'left': 189, 'width': 69, 'height': 69}, 'landmark': {'contour_chin': {'x': 242, 'y': 140}, 'contour_left1': {'x': 188, 'y': 97}, 'contour_left2': {'x': 190, 'y': 105}, 'contour_left3': {'x': 192, 'y': 113}, 'contour_left4': {'x': 196, 'y': 121}, 'contour_left5': {'x': 201, 'y': 128}, 'contour_left6': {'x': 207, 'y': 133}, 'contour_left7': {'x': 215, 'y': 137}, 'contour_left8': {'x': 223, 'y': 141}, 'contour_left9': {'x': 232, 'y': 142}, 'contour_right1': {'x': 253, 'y': 82}, 'contour_right2': {'x': 255, 'y': 88}, 'contour_right3': {'x': 256, 'y': 95}, 'contour_right4': {'x': 256, 'y': 102}, 'contour_right5': {'x': 256, 'y': 109}, 'contour_right6': {'x': 255, 'y': 116}, 'contour_right7': {'x': 253, 'y': 123}, 'contour_right8': {'x': 251, 'y': 129}, 'contour_right9': {'x': 248, 'y': 136}, 'left_eye_bottom': {'x': 21

In [1]:
import json
import numpy as np
import cv2
import os

### p1 left point, p2 right point
def point_on_line (p1, p2, ratio):
    x = p1[0] + ratio*(p2[0]-p1[0])
    y = p1[1] + ratio*(p2[1]-p1[1])
    return (int(x), int(y))

def get_ratio (p1, p2, p3):
    return (p3[0]-p1[0])/(p2[0]-p1[0])

def closest_point_on_line(p1, p2, p3):
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    dx, dy = x2-x1, y2-y1
    det = dx*dx + dy*dy
    a = (dy*(y3-y1)+dx*(x3-x1))/det
    return (int(x1+a*dx), int(y1+a*dy))

### Create map of offset from reference point
def offset_map (landmarks, ref_pt):
    off_map = {}
    for feature in landmarks.keys():
        pt= (landmarks[feature]['x'], landmarks[feature]['y'])
        off_map[feature] = (pt[0] - ref_pt[0], pt[1] - ref_pt[1])
    return off_map

### Select Image
image_path = "pic2.jpg"
image_name= os.path.splitext(image_path)[0]
json_file=image_name + ".json"
with open(image_name + ".json") as f:
    data = json.load(f)
faces = data['faces']
img = cv2.imread(image_path)

# ### Mark all points on face
# for face in faces:
#     landmarks = face['landmark']
#     for feature in landmarks.keys():
#         x=landmarks[feature]['x']
#         y=landmarks[feature]['y']
#         img = cv2.circle(img, (x,y), radius=2, color=(0, 0, 255), thickness=-1)
        
# ### Line on bottom nose           
# # for face in faces:
# #     landmarks = face['landmark']
# #     start = 'nose_tip'
# #     end = 'nose_contour_lower_middle'
# #     start_point = (landmarks[start]['x'], landmarks[start]['y'])
# #     end_point = (landmarks[end]['x'], landmarks[end]['y'])
# #     img = cv2.line(img, start_point, end_point, color=(0, 255, 0), thickness=1) 

# ### Marking contour lines
# for face in faces:
#     landmarks = face['landmark']
#     for i in range(1, 10):
#         start = 'contour_left' + str(i)
#         end = 'contour_right' + str(i)
#         ref = 'left_eye_pupil'
#         start_point = (landmarks[start]['x'], landmarks[start]['y'])
#         end_point = (landmarks[end]['x'], landmarks[end]['y'])
#         img = cv2.line(img, start_point, end_point, color=(0, 255, 0), thickness=1) 

        
### May need to adjust offset to be proprotional with changes in contour lines
### Shift Left Eye
for face in faces:
    landmarks = face['landmark']    
    start = 'contour_left1'; start_point = (landmarks[start]['x'], landmarks[start]['y'])
    end = 'contour_right1'; end_point = (landmarks[end]['x'], landmarks[end]['y'])
    ref = 'left_eye_pupil'; ref_point = (landmarks[ref]['x'], landmarks[ref]['y'])
    new_ref = closest_point_on_line(start_point, end_point, ref_point)
    
    new_start = 'contour_left3'; new_start_point = (landmarks[new_start]['x'], landmarks[new_start]['y'])
    new_end = 'contour_right3'; new_end_point = (landmarks[new_end]['x'], landmarks[new_end]['y'])
    new_ref =  point_on_line (new_start_point, new_end_point, get_ratio(start_point, end_point, new_ref))
    img = cv2.circle(img, new_ref, radius=2, color=(255, 0, 0), thickness=-1)
    
    offsets = offset_map (landmarks, ref_point)
    for feature in landmarks.keys():
        if "left_eye" in feature:
            offset = offsets[feature]
            new_pt = (new_ref[0] + offset[0], new_ref[1] + offset[1])
            img = cv2.circle(img, new_pt, radius=2, color=(255, 0, 0), thickness=-1)
            
### Shift Right Eye
for face in faces:
    landmarks = face['landmark']    
    start = 'contour_left1'; start_point = (landmarks[start]['x'], landmarks[start]['y'])
    end = 'contour_right1'; end_point = (landmarks[end]['x'], landmarks[end]['y'])
    ref = 'right_eye_pupil'; ref_point = (landmarks[ref]['x'], landmarks[ref]['y'])
    new_ref = closest_point_on_line(start_point, end_point, ref_point)
    
    new_start = 'contour_left3'; new_start_point = (landmarks[new_start]['x'], landmarks[new_start]['y'])
    new_end = 'contour_right3'; new_end_point = (landmarks[new_end]['x'], landmarks[new_end]['y'])
    new_ref =  point_on_line (new_start_point, new_end_point, get_ratio(start_point, end_point, new_ref))
    img = cv2.circle(img, new_ref, radius=2, color=(255, 0, 0), thickness=-1)
    
    offsets = offset_map (landmarks, ref_point)
    for feature in landmarks.keys():
        if "right_eye" in feature:
            offset = offsets[feature]
            new_pt = (new_ref[0] + offset[0], new_ref[1] + offset[1])
            img = cv2.circle(img, new_pt, radius=2, color=(255, 0, 0), thickness=-1)
            
### Shift mouth            
for face in faces:
    landmarks = face['landmark']    
    start = 'contour_left5'; start_point = (landmarks[start]['x'], landmarks[start]['y'])
    end = 'contour_right5'; end_point = (landmarks[end]['x'], landmarks[end]['y'])
    ref = 'mouth_upper_lip_top'; ref_point = (landmarks[ref]['x'], landmarks[ref]['y'])
    new_ref = closest_point_on_line(start_point, end_point, ref_point)
    
    new_start = 'contour_left4'; new_start_point = (landmarks[new_start]['x'], landmarks[new_start]['y'])
    new_end = 'contour_right4'; new_end_point = (landmarks[new_end]['x'], landmarks[new_end]['y'])
    new_ref =  point_on_line (new_start_point, new_end_point, get_ratio(start_point, end_point, new_ref))
    img = cv2.circle(img, new_ref, radius=2, color=(255, 0, 0), thickness=-1)
    
    offsets = offset_map (landmarks, ref_point)
    for feature in landmarks.keys():
        if "mouth" in feature:
            offset = offsets[feature]
            new_pt = (new_ref[0] + offset[0], new_ref[1] + offset[1])
            img = cv2.circle(img, new_pt, radius=2, color=(255, 0, 0), thickness=-1)

# cv2.imwrite(image_name + "_mapping.jpg", img) 
cv2.imshow('image',img)
cv2.waitKey(0)

-1

In [19]:
import json
import numpy as np
import cv2
import os

class HumanFace:
    landmarks = {}
    def __init__(self, face):
        self.landmarks = face['landmark']  

    def findCorners(self, points_dict):
        left, top = 2147483646, 2147483646
        right, bottom = 0, 0
        for feat, coord in points_dict.items():
            if coord['x'] < left: 
                left = coord['x']
            if coord['x'] > right: 
                right = coord['x']
            if coord['y'] < top: 
                top = coord['y']
            if coord['y'] > bottom: 
                bottom = coord['y']
        return left, right, bottom, top

    def get_dict(self, keyword):
        key_list = []
        for feature in self.landmarks.keys():
            if keyword in feature:
                key_list.append(feature)
        feature_dict = {k:v for k,v in self.landmarks.items() if k in key_list}
        return feature_dict

    def get_object_region(self, body_part):
        body_dict = self.get_dict(body_part)
        left, right, bottom, top = self.findCorners(body_dict)
        region_dict = {'left': left + 1, 'right': right + 1, 'top': top, 'bottom': bottom + 10}
        return region_dict
    
#     def get_points(self):
        
### Create map of offset from reference point
def get_offset (points, ref_pt):
    offsets = []
    for pt in points:
        offsets.append((pt[0] - ref_pt[0], pt[1] - ref_pt[1]))
    return offsets
    
### Select Image
image_path = "pic2.jpg"
image_name= os.path.splitext(image_path)[0]
json_file=image_name + ".json"
with open(image_name + ".json") as f:
    data = json.load(f)
faces = data['faces']
img = cv2.imread(image_path)
    
### Shift Left Eye
for face in faces:
    subject = HumanFace(face) 
    left_eye_region = subject.get_object_region('left_eye')
    point1 = (left_eye_region['left'], left_eye_region['top'])
    point2 = (left_eye_region['right'], left_eye_region['bottom'])
    ### point1[0] : left | point1[1] : top | point2[0] : right | point2[1] : bottom
    points = [point1, point2]
    
    start = 'contour_left1'; start_point = (landmarks[start]['x'], landmarks[start]['y'])
    end = 'contour_right1'; end_point = (landmarks[end]['x'], landmarks[end]['y'])
    ref = 'left_eye_pupil'; ref_point = (landmarks[ref]['x'], landmarks[ref]['y'])
    new_ref = closest_point_on_line(start_point, end_point, ref_point)
    
    new_start = 'contour_left3'; new_start_point = (landmarks[new_start]['x'], landmarks[new_start]['y'])
    new_end = 'contour_right3'; new_end_point = (landmarks[new_end]['x'], landmarks[new_end]['y'])
    new_ref =  point_on_line (new_start_point, new_end_point, get_ratio(start_point, end_point, new_ref))
    
    offsets = get_offset (points, ref_point)
    offset1 = offsets[0]
    new_point1 = (new_ref[0] + offset1[0], new_ref[1] + offset1[1])
    offset2 = offsets[1]
    new_point2 = (new_ref[0] + offset2[0], new_ref[1] + offset2[1])
    left_eye = img[point1[1]:point2[1], point1[0]:point2[0]]
    left_eye = left_eye.copy()
    
    ### Apply Inpaint
    mask = img.copy()
    mask[:,:] = 0
    mask[point1[1]:point2[1], point1[0]:point2[0]] = 255
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    img = cv2.inpaint(img,mask,25,cv2.INPAINT_TELEA)
    
    img[new_point1[1]:new_point2[1], new_point1[0]:new_point2[0]] = left_eye
    
### Shift Right Eye
for face in faces:
    subject = HumanFace(face) 
    right_eye_region = subject.get_object_region('right_eye')
    point1 = (right_eye_region['left'], right_eye_region['top'])
    point2 = (right_eye_region['right'], right_eye_region['bottom'])
    ### point1[0] : left | point1[1] : top | point2[0] : right | point2[1] : bottom
    points = [point1, point2]
    
    start = 'contour_left1'; start_point = (landmarks[start]['x'], landmarks[start]['y'])
    end = 'contour_right1'; end_point = (landmarks[end]['x'], landmarks[end]['y'])
    ref = 'right_eye_pupil'; ref_point = (landmarks[ref]['x'], landmarks[ref]['y'])
    new_ref = closest_point_on_line(start_point, end_point, ref_point)
    
    new_start = 'contour_left3'; new_start_point = (landmarks[new_start]['x'], landmarks[new_start]['y'])
    new_end = 'contour_right3'; new_end_point = (landmarks[new_end]['x'], landmarks[new_end]['y'])
    new_ref =  point_on_line (new_start_point, new_end_point, get_ratio(start_point, end_point, new_ref))
    
    offsets = get_offset (points, ref_point)
    offset1 = offsets[0]
    new_point1 = (new_ref[0] + offset1[0], new_ref[1] + offset1[1])
    offset2 = offsets[1]
    new_point2 = (new_ref[0] + offset2[0], new_ref[1] + offset2[1])
    right_eye = img[point1[1]:point2[1], point1[0]:point2[0]]
    right_eye = right_eye.copy()
    
    ### Apply Inpaint
    mask = img.copy()
    mask[:,:] = 0
    mask[point1[1]:point2[1], point1[0]:point2[0]] = 255
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    img = cv2.inpaint(img,mask,25,cv2.INPAINT_TELEA)
    
    img[new_point1[1]:new_point2[1], new_point1[0]:new_point2[0]] = right_eye
    
### Shift Mouth
for face in faces:
    subject = HumanFace(face) 
    mouth_region = subject.get_object_region('mouth')
    point1 = (mouth_region['left'], mouth_region['top'])
    point2 = (mouth_region['right'], mouth_region['bottom'])
    ### point1[0] : left | point1[1] : top | point2[0] : right | point2[1] : bottom
    points = [point1, point2]
    
    start = 'contour_left5'; start_point = (landmarks[start]['x'], landmarks[start]['y'])
    end = 'contour_right5'; end_point = (landmarks[end]['x'], landmarks[end]['y'])
    ref = 'mouth_upper_lip_top'; ref_point = (landmarks[ref]['x'], landmarks[ref]['y'])
    new_ref = closest_point_on_line(start_point, end_point, ref_point)
    
    new_start = 'contour_left3'; new_start_point = (landmarks[new_start]['x'], landmarks[new_start]['y'])
    new_end = 'contour_right3'; new_end_point = (landmarks[new_end]['x'], landmarks[new_end]['y'])
    new_ref =  point_on_line (new_start_point, new_end_point, get_ratio(start_point, end_point, new_ref))
    
    offsets = get_offset (points, ref_point)
    offset1 = offsets[0]
    new_point1 = (new_ref[0] + offset1[0], new_ref[1] + offset1[1])
    offset2 = offsets[1]
    new_point2 = (new_ref[0] + offset2[0], new_ref[1] + offset2[1])
    mouth = img[point1[1]:point2[1], point1[0]:point2[0]]
    mouth = mouth.copy()
    
    ### Apply Inpaint
    mask = img.copy()
    mask[:,:] = 0
    mask[point1[1]:point2[1], point1[0]:point2[0]] = 255
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    img = cv2.inpaint(img,mask,25,cv2.INPAINT_TELEA)
    
    img[new_point1[1]:new_point2[1], new_point1[0]:new_point2[0]] = mouth
    
    
    
cv2.imwrite(image_name + "_output.jpg", img) 
cv2.imshow('image',img)
cv2.waitKey(0)
            

-1